In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments,Trainer, pipeline
from datasets import load_dataset
import numpy as np
import evaluate
from sklearn.metrics import classification_report

#### Downloading the base model and getting the tokenizer

In [ ]:
model_name = "google-bert/bert-base-uncased"

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

#### Loading the data set

In [ ]:
data = load_dataset('GonzaloA/fake_news')

In [ ]:
data = data.remove_columns(['Unnamed: 0','title'])

In [ ]:
tokenized_data = data.map(tokenize_function, batched=True)

#### Sampling from the data set

In [ ]:
small_train_dataset = tokenized_data["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_data["validation"].shuffle(seed=42).select(range(100))
small_test_dataset = tokenized_data['test']

#### Outputting the training arguments

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

#### Loading the evaluation metrics

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### Initializing the Trainer object and fine-tuning the model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

#### Evaluating the model

In [ ]:
test_labels = small_test_dataset['label']
small_test_dataset = small_test_dataset.remove_columns(['label','token_type_ids'])

In [ ]:
predictions = trainer.predict(small_test_dataset)

In [ ]:
predicted_labels = predictions.predictions.argmax(axis=1)

In [ ]:
print(classification_report(test_labels, predicted_labels))

#### Saving the model

In [ ]:
trainer.save_model('bert-base-uncased-fake-news-classification')

In [ ]:
tokenizer.save_pretrained('bert-base-uncased-fake-news-classification')

#### Saving the predictions

In [ ]:
with open('predictions/predictions.txt', 'w') as f:
    for line in predicted_labels:
        f.write(f"{line}\n")